In [1]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment, PatternFill, Font
from openpyxl.worksheet.dimensions import ColumnDimension
from datetime import datetime, timedelta

# Import csv
df_company = pd.read_csv('mock_company_data_1-5Sep2024.csv')
df_bank = pd.read_csv('mock_bank_data_1-5Sep2024.csv')

In [2]:
# Define start_date, end_date, month, and year
start_date = 1
end_date = 5
month = 9 
year = 2024

In [3]:
# Check data info
df_company.info()
df_company.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99997 entries, 0 to 99996
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   transaction_id  99997 non-null  object
 1   date            99997 non-null  object
 2   amount          99997 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99995 entries, 0 to 99994
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   transaction_id  99995 non-null  object
 1   date            99995 non-null  object
 2   amount          99995 non-null  int64 
 3   ref_code        99995 non-null  object
dtypes: int64(1), object(3)
memory usage: 3.1+ MB


,transaction_id,date,amount,ref_code
0,TR_240902_00001,2/9/2024 13:02,458,Z008
1,TR_240905_00002,5/9/2024 18:57,478,Z001
2,TR_240902_00003,2/9/2024 13:02,263,Z007
3,TR_240903_00004,3/9/2024 16:31,332,Z007
4,TR_240902_00005,2/9/2024 13:02,289,Z003


In [ ]:
df_bank.info()
df_bank.head()

In [4]:
# Change datatype
df_company['datetime'] = pd.to_datetime(df_company['datetime'], format='%d/%m/%Y %H:%M')
df_bank['datetime'] = pd.to_datetime(df_bank['datetime'], format='%d/%m/%Y %H:%M')

# Sort data by datetime
df_company = df_company.sort_values(by='datetime')
df_bank = df_bank.sort_values(by='datetime')

# Add a helper column to keep only 'date' value
df_company['date_formatted'] = df_company['datetime'].dt.strftime('%Y-%m-%d')
df_bank['date_formatted'] = df_bank['datetime'].dt.strftime('%Y-%m-%d')

KeyError: 'datetime'

In [ ]:
# Find Unmatch Transactions
# Merge the two DataFrames on 'transaction_id'
fulldf = pd.merge(df_company, df_bank, on='transaction_id', how='outer', suffixes=('_company', '_bank'))
fulldf['Diff Amount'] = fulldf['amount_company'].fillna(0) - fulldf['amount_bank'].fillna(0)

# Define conditions for 'reconciliation_status'
conditions = [
    (fulldf['transaction_id'].notna() & fulldf['amount_company'].notna() & 
     fulldf['amount_bank'].notna() & (fulldf['amount_company'] == fulldf['amount_bank'])),
    (fulldf['transaction_id'].notna() & fulldf['amount_company'].notna() & 
     fulldf['amount_bank'].notna() & (fulldf['amount_company'] != fulldf['amount_bank'])),
    (fulldf['transaction_id'].notna() & fulldf['amount_company'].notna() & 
     fulldf['amount_bank'].isna()),
    (fulldf['transaction_id'].notna() & fulldf['amount_company'].isna() & 
     fulldf['amount_bank'].notna())
]

# Define the corresponding values for 'reconciliation_status'
choices = ['match data', 'diff amount', 'found in company only', 'found in bank only']

# Create the 'reconciliation_status' column
fulldf['reconciliation_status'] = np.select(conditions, choices, default='unknown')

df_unmatch = fulldf[fulldf['reconciliation_status'] != 'match data'].copy()

In [ ]:
# Create a workbook and add sheets
wb = Workbook()

# 1. Summary Sheet
ws_summary = wb.active
ws_summary.title = "Summary"

# Add "Total" in C2 & Apply formatting
ws_summary["C2"] = "Total"
ws_summary["C2"].fill = PatternFill(start_color="FFFF99", end_color="FFFF99", fill_type="solid")
ws_summary["C2"].font = Font(bold=True)
ws_summary["C2"].alignment = Alignment(horizontal="center") 

# Add dates in D2 onward & Apply formatting
start_date_obj = datetime(year, month, start_date)
for i in range(end_date - start_date + 1):
    date = start_date_obj + timedelta(days=i)
    cell = ws_summary.cell(row=2, column=4 + i)
    cell.value = date.strftime('%Y-%m-%d')
    cell.font = Font(bold=True)
    cell.alignment = Alignment(horizontal="center")

# Add "Company Amount " and "Bank Amount" labels in B3 and B4 & Apply formatting
data = [("B3", "Company Amount"), ("B4", "Bank Amount")]
for cell, value in data:
    ws_summary[cell] = value
    ws_summary[cell].font = Font(bold=True)

# Add "Diff" labels in B5 & Apply formatting
ws_summary["B5"] = "Diff"
ws_summary["B5"].font = Font(bold=True)
ws_summary["B5"].font = Font(bold=True, color='FF0000')

# Add SUMIF formulas in D3, D4 and D5 onward
last_column = 4 + (end_date - start_date)
for col in range(4, last_column + 1):  
    current_col_letter = ws_summary.cell(row=2, column=col).column_letter
    ws_summary.cell(row=3, column=col).value = f"=SUMIF('Company Data'!D:D, {current_col_letter}2, 'Company Data'!C:C)"
    ws_summary.cell(row=4, column=col).value = f"=SUMIF('Bank Data'!E:E, {current_col_letter}2, 'Bank Data'!C:C)"
    ws_summary.cell(row=5, column=col).value = f"={current_col_letter}3-{current_col_letter}4"

# Add total formulas in C3, C4, and C5 onward
last_col_letter = ws_summary.cell(row=2, column=last_column).column_letter
data = [("C3", f"=SUM(D3:{last_col_letter}3)"), ("C4", f"=SUM(D4:{last_col_letter}4)")]
for cell, value in data:
    ws_summary[cell] = value
    ws_summary[cell].font = Font(bold=True)
    ws_summary[cell].alignment = Alignment(horizontal="center")

ws_summary["C5"] = f"=SUM(D5:{last_col_letter}5)"
ws_summary["C5"].font = Font(bold=True, color='FF0000')
ws_summary["C5"].alignment = Alignment(horizontal="center")

# Apply number formatting in Summary Sheet
for col in range(3, last_column + 1):
    ws_summary.cell(row=3, column=col).number_format = '#,##0.00'
    ws_summary.cell(row=4, column=col).number_format = '#,##0.00'
    ws_summary.cell(row=5, column=col).number_format = '#,##0.00'

# 2. Company Data Sheet
ws_company = wb.create_sheet(title="Company Data")

# Add data from the DataFrame & Apply formatting for the amount column
for r_idx, row in enumerate(dataframe_to_rows(df_company, index=False, header=True), start=1):
    for c_idx, value in enumerate(row, start=1):
        cell = ws_company.cell(row=r_idx, column=c_idx, value=value)
        if isinstance(value, (int, float)): 
            cell.number_format = '#,##0.00'

# 3. Bank Data Sheet
ws_bank = wb.create_sheet(title="Bank Data")

# Add data from the DataFrame & Apply formatting for the amount column
for r_idx, row in enumerate(dataframe_to_rows(df_bank, index=False, header=True), start=1):
    for c_idx, value in enumerate(row, start=1):
        cell = ws_bank.cell(row=r_idx, column=c_idx, value=value)
        if isinstance(value, (int, float)): 
            cell.number_format = '#,##0.00'

# 4. Unmatch Details Sheet
ws_unmatch = wb.create_sheet(title="Unmatch Details")
for r_idx, row in enumerate(dataframe_to_rows(df_unmatch, index=False, header=True), start=1):
    for c_idx, value in enumerate(row, start=1):
        cell = ws_unmatch.cell(row=r_idx, column=c_idx, value=value)
        if isinstance(value, (int, float)): 
            cell.number_format = '#,##0.00'

# Adjust Column Width of Summary Sheet
for idx, column_cells in enumerate(ws_summary.columns, 1):
    if idx > 1:  # Skip column A (index 1)
        ws_summary.column_dimensions[column_cells[0].column_letter].width = 15

# Adjust Column Width of Company Sheet, Bank Sheet, Unmatch Details Sheet
for ws in [ws_company, ws_bank, ws_unmatch]:
    for column_cells in ws.columns:
        # Find the maximum length of the content in each column
        max_length = max(len(str(cell.value)) for cell in column_cells if cell.value is not None)
        
        # Adjust the width, adding a small buffer for padding
        adjusted_width = max_length + 2  # You can tweak this value for more padding
        
        # Set the column width using the column letter of the first cell in the column
        ws.column_dimensions[column_cells[0].column_letter].width = adjusted_width

# Generate filename based on the date, month, year variables
month_abbr = datetime(year, month, 1).strftime('%b')
filename = f"Reconciliation_report_{start_date}-{end_date}_{month_abbr}_{year}.xlsx"

# Save the workbook
wb.save(filename)

print(f"Reconciliation report has been created successfully as {filename}.")